<a href="https://colab.research.google.com/github/norhan333/NeuroPhone-RealTime-BrainMobilePhone-Interface/blob/main/model/EPFL_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install -q MOABB

['\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/140.8 KB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━\x1b \x1b133.1/140.8 KB\x1b \x1b4.7 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b140.8/140.8 KB\x1b \x1b3.3 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/90.6 KB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b90.6/90.6 KB\x1b \x1b3.5 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h  Preparing metadata (setup.py) ... \x1b[?25l\x1b[?25hdone',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/662.4 KB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b662.4/662.4 KB\x1b \x1b17.3 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pywd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pywd: filename=pywd-0.0.3-py3-none-any.whl size=2238 sha256=2d3a5af89b77cfefab9d22e27dc008ba7435d854fa3d7064885ab5ec41889535
  Stored in directory: /root/.cache/pip/wheels/b5/7a/5f/1512c6efc8bed9ef6a8c12ffea89c0b879e3dd195043cb999a
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=c9ba3f688274211a37379f6eff879d981824174e69b2219f91cae1d9bad4ea67
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built pywd docopt


In [ ]:
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [ ]:
import moabb
from moabb.datasets import EPFLP300, utils
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.pipelines.features import LogVariance
from mne.filter import construct_iir_filter
moabb.set_log_level("info")


In [ ]:
from pywt import wavedec 

In [ ]:
dataset=EPFLP300()
dataset._get_single_subject_data(1)['session_1']['run_1']

/usr/local/lib/python3.8/dist-packages/moabb/datasets/download.py:53: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_EPFLP300_PATH"
  set_config(key, get_config("MNE_DATA"))


MNE_DATA is not already configured. It will be set to default location in the home directory - /root/mne_data
All datasets will be downloaded to this location, if anything is already downloaded, please move manually to this location


100%|███████████████████████████████████████| 267M/267M [00:00<00:00, 97.1GB/s]
SHA256 hash of downloaded file: a493408349f2b5fc1a94f0bb5ed3f6cdcb2cdb290af7cb4c118cbc65c7ee1128
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


unzip /root/mne_data/MNE-epflp300-data/groups/m/mm/mmspg/www/BCI/p300/subject1.zip


<RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>

In [ ]:
import glob
import os
import zipfile
from moabb.datasets import download as dl
EPFLP300_URL = "http://documents.epfl.ch/groups/m/mm/mmspg/www/BCI/p300/"
def data_path(
        self, subject, path=None, force_update=False, update_path=None, verbose=None
    ):

        if subject not in self.subject_list:
            raise (ValueError("Invalid subject number"))

        # check if has the .zip
        url = "{:s}subject{:d}.zip".format(EPFLP300_URL, subject)
        path_zip = dl.data_dl(url, "EPFLP300")
        path_folder = path_zip.strip("subject{:d}.zip".format(subject))

        # check if has to unzip
        if not (os.path.isdir(path_folder + "subject{:d}".format(subject))):
            print("unzip", path_zip)
            zip_ref = zipfile.ZipFile(path_zip, "r")
            zip_ref.extractall(path_folder)

        # get the path to all files
        pattern = os.path.join("subject{:d}".format(subject), "*", "*")
        subject_paths = glob.glob(path_folder + pattern)

        return subject_paths


In [ ]:
from scipy.io import loadmat
file_path_list = dataset.data_path(1)
sessions = {}

for file_path in sorted(file_path_list):

    session_name = "session_" + file_path.split(os.sep)[-2].replace("session", "")

    if session_name not in sessions.keys():
        sessions[session_name] = {}

    run_name = "run_" + str(len(sessions[session_name]) + 1)
    sessions[session_name][run_name] = dataset._get_single_run_data(file_path)


    data = loadmat(file_path)
    signals = data["data"]
    stimuli = data["stimuli"].squeeze()
    events = data["events"]
    target = data["target"][0][0]
    print(stimuli)
    break

[1 3 4 5 2 6 2 3 6 1 5 4 5 2 6 3 4 1 6 2 5 1 4 3 5 2 4 1 6 3 5 2 1 3 6 4 6
 1 5 3 4 2 6 1 2 4 5 3 2 6 5 1 4 3 6 3 5 4 2 1 1 5 2 3 4 6 2 5 4 1 6 3 6 4
 1 2 5 3 6 3 2 5 4 1 1 3 6 4 5 2 2 5 1 6 4 3 3 1 2 6 4 5 2 3 6 1 5 4 5 3 6
 1 4 2 3 1 4 2 5 6 4 6 2 3 5 1 5 3 6 2 4 1 6 3 5 2 4 1]


In [ ]:
def getData(subjectNumbers,dataset=EPFLP300()):
  return dataset.get_data(subjectNumbers)

In [ ]:
def getSubjectData(subjectNumber,session_number,run_number):
  session = f'session_{session_number}'
  run = f'run_{run_number}'
  return getData([subjectNumber])[subjectNumber][session][run]

In [ ]:
getSubjectData(1,1,1)

<RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>

In [ ]:
data= getSubjectData(1,1,1)

In [ ]:
data.get_data

<bound method get_data of <RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>>

Count of Stimulus in samples

In [ ]:
df=data.to_data_frame()
df.STI.value_counts()

0.0    114549
1.0       115
2.0        23
Name: STI, dtype: int64

Index of First flash (Stimulus)

In [ ]:
df.STI.ne(0).idxmax()

819

In [ ]:
df.iloc[819]

time        0.399902
Fp1      3578.843750
AF3     27160.750000
F7      -9499.781250
F3      19571.750000
FC1      1656.812500
FC5     -1478.093750
T7      -8032.781250
C3       3472.593750
CP1    -12703.375000
CP5    -11995.437500
P7       2006.437500
P3      -7322.250000
Pz      11253.906250
PO3     -2716.781250
O1        223.062500
Oz       -989.312500
O2       2215.562500
PO4     -6220.125000
P4      -6175.562500
P8      -5917.625000
CP6     -3828.718750
CP2     -4155.562500
C4       7376.375000
T8       8960.312500
FC6     -4080.625000
FC2      7104.343750
F4      -2386.750000
F8     -16126.531250
AF4      4861.500000
Fp2     22986.468750
Fz       6994.093750
Cz       8327.312500
MA1         0.006691
MA2        -0.006691
STI         1.000000
Name: 819, dtype: float64

Get Sample indicies of Target stimulus

In [ ]:
Target = df[df['STI']==2].index.values
val = np.full(len(Target),2)
see= pd.DataFrame(Target)
see["value"]=val
see

,0,value
0,4099,2
1,5740,2
2,11482,2
3,16406,2
4,21327,2
5,26251,2
6,34453,2
7,36915,2
8,40197,2
9,48400,2


Get Sample indicies of Non-Target stimulus

In [ ]:
nonTarget = df[df['STI']==1].index.values
value = np.full(len(nonTarget),1)
dataf= pd.DataFrame(nonTarget)
dataf["value"]=value
dataf

,0,value
0,819,1
1,1636,1
2,2459,1
3,3280,1
4,4921,1
...,...,...
110,109109,1
111,109950,1
112,110749,1
113,112389,1


Combine and sort (based on time) target and non-target samples

In [ ]:
df2 = dataf.append(see,ignore_index=True) 
TargetNnonTarget = df2.sort_values(by=[0],ignore_index=True)
TargetNnonTarget

,0,value
0,819,1
1,1636,1
2,2459,1
3,3280,1
4,4099,2
...,...,...
133,109950,1
134,110749,1
135,111570,2
136,112389,1


Divide each trail (flash) into segments and targets

In [ ]:
segments=[]
output=[]
length=len(TargetNnonTarget)
for index, row in TargetNnonTarget.iterrows():
  if(index==length-1):
    Segment = df.iloc[TargetNnonTarget[0].iloc[index]:TargetNnonTarget[0].iloc[index]+820]
  else:
    Segment = df.iloc[TargetNnonTarget[0].iloc[index]:TargetNnonTarget[0].iloc[index+1]]
  segments.append(Segment)
  output.append(TargetNnonTarget['value'].iloc[index])


Print length of each Trail

In [ ]:
for seg in segments:
  print(len(seg))

In [ ]:
df

,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.000000,3566.625000,27146.750000,-9512.343750,19556.281250,1640.281250,-1500.781250,-8068.000000,3436.687500,-12713.562500,...,7099.187500,-2389.000000,-16210.187500,4854.968750,22967.625000,6985.312500,8299.875000,0.006680,-0.006680,0.0
1,0.000488,3564.203125,27144.609375,-9515.796875,19553.046875,1638.640625,-1502.734375,-8072.609375,3434.109375,-12718.734375,...,7095.828125,-2392.078125,-16213.421875,4852.578125,22965.609375,6981.921875,8297.671875,0.006677,-0.006677,0.0
2,0.000977,3561.390625,27141.890625,-9519.296875,19548.890625,1635.734375,-1504.921875,-8076.140625,3433.578125,-12722.203125,...,7092.609375,-2395.671875,-16216.046875,4850.046875,22964.734375,6980.421875,8295.234375,0.006671,-0.006671,0.0
3,0.001465,3559.515625,27139.015625,-9521.640625,19546.609375,1632.765625,-1505.546875,-8077.828125,3430.984375,-12722.890625,...,7089.953125,-2398.734375,-16219.546875,4845.796875,22962.515625,6979.203125,8293.765625,0.006667,-0.006667,0.0
4,0.001953,3556.421875,27137.296875,-9523.734375,19546.296875,1631.390625,-1507.171875,-8081.390625,3430.109375,-12724.953125,...,7087.453125,-2400.859375,-16222.421875,4842.890625,22960.390625,6977.453125,8291.078125,0.006665,-0.006665,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114682,55.997070,3530.156250,27568.593750,-10146.500000,19951.156250,1380.687500,-455.531250,-8331.031250,3226.156250,-12875.937500,...,6766.250000,-2992.875000,-17246.656250,4532.218750,23213.656250,6191.156250,8356.093750,0.006414,-0.006414,0.0
114683,55.997559,3526.687500,27566.281250,-10151.531250,19947.906250,1376.312500,-459.531250,-8334.875000,3221.906250,-12879.031250,...,6761.750000,-2997.468750,-17251.468750,4530.093750,23209.468750,6186.343750,8352.718750,0.006414,-0.006414,0.0
114684,55.998047,3522.703125,27564.140625,-10153.359375,19945.796875,1372.734375,-462.015625,-8337.953125,3217.453125,-12883.265625,...,6758.140625,-2999.609375,-17260.046875,4524.984375,23207.359375,6182.796875,8349.890625,0.006413,-0.006413,0.0
114685,55.998535,3524.187500,27564.968750,-10151.437500,19946.687500,1373.281250,-461.093750,-8337.000000,3218.843750,-12884.906250,...,6759.375000,-3000.281250,-17261.968750,4524.187500,23207.906250,6183.468750,8349.250000,0.006412,-0.006412,0.0


In [ ]:
df.iloc[819:1636]

,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
819,0.399902,3578.843750,27160.750000,-9499.781250,19571.750000,1656.812500,-1478.093750,-8032.781250,3472.593750,-12703.375000,...,7104.343750,-2386.750000,-16126.531250,4861.500000,22986.468750,6994.093750,8327.312500,0.006691,-0.006691,1.0
820,0.400391,3580.281250,27162.906250,-9501.156250,19572.312500,1659.125000,-1477.375000,-8034.281250,3475.562500,-12700.312500,...,7107.187500,-2384.531250,-16124.937500,4864.062500,22988.406250,6995.781250,8329.437500,0.006690,-0.006690,0.0
821,0.400879,3579.812500,27163.531250,-9504.281250,19571.625000,1657.500000,-1477.343750,-8036.531250,3474.562500,-12700.750000,...,7106.375000,-2383.875000,-16123.906250,4862.812500,22987.531250,6994.718750,8329.593750,0.006686,-0.006686,0.0
822,0.401367,3578.031250,27161.687500,-9504.468750,19570.875000,1657.125000,-1476.156250,-8037.531250,3472.687500,-12700.312500,...,7105.406250,-2385.812500,-16123.375000,4860.250000,22986.843750,6992.281250,8329.281250,0.006682,-0.006682,0.0
823,0.401855,3576.703125,27159.703125,-9504.328125,19570.015625,1656.578125,-1475.703125,-8037.328125,3472.015625,-12699.078125,...,7105.140625,-2387.078125,-16123.203125,4858.703125,22985.453125,6992.453125,8327.359375,0.006680,-0.006680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,0.796387,3572.031250,27157.031250,-9508.906250,19567.750000,1649.625000,-1492.531250,-8058.718750,3470.250000,-12714.156250,...,7115.062500,-2388.343750,-16168.218750,4867.843750,22991.468750,6996.625000,8339.375000,0.006688,-0.006688,0.0
1632,0.796875,3567.218750,27151.312500,-9506.656250,19557.687500,1641.718750,-1519.406250,-8058.281250,3458.906250,-12719.906250,...,7109.375000,-2393.468750,-16171.781250,4864.968750,22986.937500,6989.593750,8333.687500,0.006679,-0.006679,0.0
1633,0.797363,3566.906250,27150.156250,-9506.343750,19552.875000,1637.062500,-1526.281250,-8056.562500,3452.281250,-12719.625000,...,7108.312500,-2392.187500,-16171.468750,4864.968750,22985.250000,6989.156250,8331.562500,0.006679,-0.006679,0.0
1634,0.797852,3574.765625,27157.671875,-9502.078125,19562.796875,1645.234375,-1513.640625,-8053.265625,3460.421875,-12711.546875,...,7118.265625,-2383.046875,-16162.484375,4872.984375,22993.078125,6998.078125,8339.265625,0.006689,-0.006689,0.0


In [ ]:
def filterData(data,lp=12,hp=1,sf=2048,resample=32):
  iir_params = dict(order=4, ftype='butter', output='sos')  
  iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
  data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
  return data

In [ ]:
lp=12;hp=1;sf=2048;resample=32
iir_params = dict(order=4, ftype='butter', output='sos')  
iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
data=getSubjectData(1,1,1)
data
# x=data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
# x

<RawArray | 35 x 114687 (56.0 s), ~30.7 MB, data loaded>

In [ ]:
subjects={}
for subjectNumber in range(1,9):
  sessions={}
  for sessionNumber in range(1,5):
    runs={}
    for runNumber in range(1,7):
      if(subjectNumber==5):
        continue
      data = getSubjectData(subjectNumber,sessionNumber,runNumber)
      print(data)
      data = filterData(data)
      print(data)
      runs[runNumber] = data.resample(32)
      print(runs[runNumber])
    sessions[sessionNumber]=runs
  subjects[subjectNumber]=sessions

In [ ]:
def bandPassFilter(data, sampligFrequency, lowFrequency, highFrequency, order):
  sos=scipy.signal.butter(order, Wn=[lowFrequency,highFrequency],btype='bandpass', fs=sampligFrequency, output='sos')
  filteredData = scipy.signal.sosfilt(sos, data)
 #plotting
  t = np.linspace(0, 1, 114687 , False)  # 1 second
  fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
  ax1.plot(t, filteredData)
  ax1.set_title('10 Hz and 20 Hz sinusoids')
  ax1.axis([0, 1,1, 20])

  ax2.plot(t, filteredData)
  ax2.set_title('After 15 Hz high-pass filter')
  ax2.axis([0, 1,1, 20])
  ax2.set_xlabel('Time [seconds]')
  plt.tight_layout()
  plt.show()
  return filteredData

In [ ]:
df = subjects[1][1][1].to_data_frame()
df.head()

,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.00000,0.000695,0.005022,0.010937,0.000722,-0.006139,0.009083,0.004176,0.000725,0.015054,...,-0.011320,-0.008231,0.029393,-0.013295,0.003317,-0.007605,0.003051,0.006680,-0.006680,0.0
1,0.03125,1.721186,1.962593,0.244127,2.630021,6.287801,12.691410,3.095202,8.093628,-6.216290,...,0.901251,3.852332,-7.232220,3.423578,3.188274,1.634911,2.264884,0.006667,-0.006667,0.0
2,0.06250,3.122874,2.398689,0.720184,3.597990,8.500251,14.986791,8.393488,9.389298,-8.420373,...,-0.780711,0.310387,-20.997922,3.705587,1.565788,4.075645,2.900440,0.006672,-0.006672,0.0
3,0.09375,1.914845,0.020576,0.209722,-0.144635,3.516894,4.884602,9.688516,3.144019,-5.922161,...,-5.292389,-4.037025,-16.890500,0.544083,0.727026,0.317413,3.553329,0.006671,-0.006671,0.0
4,0.12500,-0.232466,-3.972272,-0.302832,-5.573326,-2.391631,1.293344,6.990735,0.081583,-5.379548,...,-6.467250,-3.905312,-2.239258,-1.778435,-0.820241,-6.466986,1.488497,0.006676,-0.006676,0.0


In [ ]:
values = df.loc[0].values
values

array([ 0.        ,  0.00069461,  0.00502216,  0.01093695,  0.00072234,
       -0.00613907,  0.0090826 ,  0.00417584,  0.00072545,  0.01505445,
        0.02245657,  0.00333073,  0.00179691, -0.00776365, -0.0067918 ,
       -0.00796106, -0.00375038, -0.00753582, -0.00966181, -0.01317375,
       -0.00335871, -0.02691801, -0.02515232, -0.00914003, -0.02298309,
       -0.00967572, -0.01131983, -0.0082305 ,  0.02939334, -0.01329538,
        0.00331689, -0.00760492,  0.00305117,  0.00667997, -0.00667997,
        0.        ])

In [ ]:
def scaleSubjectData(subjectData):
    scaler = MinMaxScaler(feature_range = (-1,1))
    df = subjects[1][1][1].to_data_frame()
    columns = ['time', 'Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1',
       'CP5', 'P7', 'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8',
       'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz',
       'Cz', 'MA1', 'MA2', 'STI']
    time = df['time']
    df.drop(columns = ['time'],axis=1,inplace=True)
    for entry in range(len(df)):
       values = df.loc[entry].values
       values = values.reshape(35,1)
       scaledRow = scaler.fit_transform(values)
       df.loc[entry] = scaledRow.reshape(35,)
    df['time'] = time
    df = df[columns]
    return df

In [ ]:
scaledDataFrame = scaleSubjectData(subjects[1][1][1])
scaledDataFrame.head()

,time,Fp1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,...,FC2,F4,F8,AF4,Fp2,Fz,Cz,MA1,MA2,STI
0,0.00000,-0.019288,0.134413,0.344488,-0.018303,-0.261998,0.278627,0.104354,-0.018192,0.490728,...,-0.446003,-0.336279,1.000000,-0.516168,0.073847,-0.314061,0.064410,0.193293,-0.281209,-0.043958
1,0.03125,0.092554,0.112523,-0.029627,0.167732,0.470300,1.000000,0.206211,0.619676,-0.564027,...,0.024729,0.268840,-0.648063,0.233374,0.213910,0.085417,0.137528,-0.049270,-0.050373,-0.049821
2,0.06250,0.363350,0.324488,0.234415,0.388846,0.651915,1.000000,0.646185,0.699623,-0.256092,...,0.153873,0.212424,-0.931037,0.394620,0.279793,0.414478,0.351414,0.196126,0.195410,0.195768
3,0.09375,0.415052,0.272513,0.286746,0.260082,0.535602,0.638518,1.000000,0.507544,-0.174662,...,-0.127273,-0.032810,-1.000000,0.311906,0.325672,0.294849,0.538344,0.271467,0.270463,0.270965
4,0.12500,0.076277,-0.359833,0.068072,-0.546537,-0.175510,0.254207,0.918597,0.112899,-0.523940,...,-0.650781,-0.352025,-0.157741,-0.104003,0.007735,-0.650750,0.276964,0.104164,0.102607,0.103386


In [ ]:
scaledValues = scaledDataFrame.loc[0].values
scaledValues

array([ 0.        , -0.01928762,  0.13441306,  0.34448756, -0.01830266,
       -0.26199811,  0.27862707,  0.10435444, -0.01819243,  0.49072813,
        0.75362794,  0.07433878,  0.01986257, -0.31969799, -0.28518117,
       -0.3267097 , -0.17715951, -0.31160626, -0.38711467, -0.51184787,
       -0.16324864, -1.        , -0.93728824, -0.36858281, -0.86024408,
       -0.38760887, -0.44600251, -0.33627923,  1.        , -0.51616768,
        0.07384732, -0.3140607 ,  0.06440982,  0.19329332, -0.28120932,
       -0.043958  ])

In [ ]:
def discrete_wavelet_transform(dataFrame):
   featureVector = []
   try: 
     dataFrame.drop(columns=['time'],inplace=True)
   except:
     pass
   for column in dataFrame.columns:
     cA2, cD2, cD1 = wavedec(dataFrame[column],'db1',level=2)
     featureVector.extend(cD2)
     featureVector.extend(cD1)
   return featureVector